# 📊 Análisis Exploratorio de Proyectos de Ley - Congreso del Perú

Este notebook contiene el análisis exploratorio de los proyectos de ley extraídos del Congreso del Perú.

## 📋 Contenido
1. **Carga y Limpieza de Datos**
2. **Análisis Descriptivo**
3. **Análisis Temporal**
4. **Análisis por Partido Político**
5. **Análisis por Tipo de Proyecto**
6. **Análisis de Autores**
7. **Visualizaciones Interactivas**


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import glob
import os
import sys

# Agregar el directorio padre al path para importar utils
sys.path.append('..')
from utils.limpieza import DataCleaner

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurar pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de análisis: {datetime.now().strftime('%d/%m/%Y %H:%M')}")


## 1. 📂 Carga y Limpieza de Datos


In [ ]:
# Cargar todos los archivos CSV disponibles
data_files = glob.glob('../data/proyectos_ley_*.csv')
print(f"📁 Archivos encontrados: {len(data_files)}")
for file in data_files:
    print(f"   - {file}")

# Cargar y combinar todos los datos
dataframes = []
for file in data_files:
    try:
        df = pd.read_csv(file, encoding='utf-8-sig')
        df['archivo_origen'] = os.path.basename(file)
        dataframes.append(df)
        print(f"✅ Cargado: {file} ({len(df)} registros)")
    except Exception as e:
        print(f"❌ Error cargando {file}: {e}")

# Combinar todos los dataframes
if dataframes:
    df_raw = pd.concat(dataframes, ignore_index=True)
    print(f"\n📊 Total de registros combinados: {len(df_raw)}")
    print(f"📅 Columnas disponibles: {list(df_raw.columns)}")
else:
    print("❌ No se encontraron archivos de datos")
    df_raw = pd.DataFrame()


In [ ]:
# Aplicar limpieza de datos
if not df_raw.empty:
    print("🧹 Aplicando limpieza de datos...")
    cleaner = DataCleaner()
    df_clean = cleaner.limpiar_dataframe(df_raw)
    
    print(f"✅ Limpieza completada")
    print(f"📊 Registros después de limpieza: {len(df_clean)}")
    
    # Mostrar primeras filas
    print("\n🔍 Primeras 5 filas de datos limpios:")
    display(df_clean.head())
    
    # Mostrar información básica
    print(f"\n📋 Información del dataset:")
    print(f"   - Forma: {df_clean.shape}")
    print(f"   - Memoria utilizada: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   - Valores nulos por columna:")
    null_counts = df_clean.isnull().sum()
    for col, count in null_counts[null_counts > 0].items():
        print(f"     * {col}: {count}")
else:
    print("❌ No hay datos para limpiar")
    df_clean = pd.DataFrame()


## 2. 📊 Análisis Descriptivo


In [ ]:
# Generar resumen estadístico
if not df_clean.empty:
    resumen = cleaner.generar_resumen(df_clean)
    
    print("📊 RESUMEN ESTADÍSTICO")
    print("=" * 50)
    print(f"📈 Total de proyectos: {resumen['total_proyectos']}")
    print(f"📅 Período: {resumen['fecha_inicio'].strftime('%d/%m/%Y')} - {resumen['fecha_fin'].strftime('%d/%m/%Y')}")
    print(f"👥 Promedio de autores por proyecto: {resumen['promedio_autores']:.1f}")
    print(f"🏆 Congresista más activo: {resumen['congresista_mas_activo']}")
    
    print(f"\n🏛️ PROYECTOS POR PARTIDO POLÍTICO:")
    for partido, count in list(resumen['proyectos_por_partido'].items())[:10]:
        print(f"   {partido}: {count}")
    
    print(f"\n📋 PROYECTOS POR TIPO:")
    for tipo, count in list(resumen['proyectos_por_tipo'].items())[:10]:
        print(f"   {tipo}: {count}")
    
    print(f"\n🌍 PROYECTOS POR REGIÓN:")
    for region, count in list(resumen['proyectos_por_region'].items())[:10]:
        print(f"   {region}: {count}")
else:
    print("❌ No hay datos para análisis")


## 3. 📈 Visualizaciones Interactivas


In [ ]:
# Crear visualizaciones interactivas
if not df_clean.empty:
    
    # 1. Proyectos por partido político
    fig1 = px.bar(
        x=list(resumen['proyectos_por_partido'].keys())[:10],
        y=list(resumen['proyectos_por_partido'].values())[:10],
        title="📊 Proyectos de Ley por Partido Político (Top 10)",
        labels={'x': 'Partido Político', 'y': 'Número de Proyectos'},
        color=list(resumen['proyectos_por_partido'].values())[:10],
        color_continuous_scale='viridis'
    )
    fig1.update_layout(
        xaxis_tickangle=-45,
        height=500,
        showlegend=False
    )
    fig1.show()
    
    # 2. Proyectos por tipo
    fig2 = px.pie(
        values=list(resumen['proyectos_por_tipo'].values()),
        names=list(resumen['proyectos_por_tipo'].keys()),
        title="📋 Distribución de Proyectos por Tipo"
    )
    fig2.update_traces(textposition='inside', textinfo='percent+label')
    fig2.show()
    
    # 3. Proyectos por región
    if resumen['proyectos_por_region']:
        fig3 = px.bar(
            x=list(resumen['proyectos_por_region'].keys())[:10],
            y=list(resumen['proyectos_por_region'].values())[:10],
            title="🌍 Proyectos de Ley por Región (Top 10)",
            labels={'x': 'Región', 'y': 'Número de Proyectos'},
            color=list(resumen['proyectos_por_region'].values())[:10],
            color_continuous_scale='plasma'
        )
        fig3.update_layout(
            xaxis_tickangle=-45,
            height=500,
            showlegend=False
        )
        fig3.show()
    
    # 4. Timeline de proyectos
    if 'fecha_datetime' in df_clean.columns:
        df_timeline = df_clean.groupby(df_clean['fecha_datetime'].dt.date).size().reset_index()
        df_timeline.columns = ['fecha', 'proyectos']
        
        fig4 = px.line(
            df_timeline,
            x='fecha',
            y='proyectos',
            title="📅 Timeline de Proyectos de Ley",
            labels={'fecha': 'Fecha', 'proyectos': 'Número de Proyectos'}
        )
        fig4.update_layout(height=400)
        fig4.show()
    
    print("✅ Visualizaciones generadas correctamente")
else:
    print("❌ No hay datos para visualizar")


## 4. 🔍 Análisis Detallado


In [ ]:
# Análisis detallado de autores y colaboraciones
if not df_clean.empty:
    
    # Análisis de autores más activos
    print("👥 ANÁLISIS DE AUTORES")
    print("=" * 40)
    
    # Contar apariciones de cada autor
    todos_autores = []
    for autores_list in df_clean['autores_limpios']:
        todos_autores.extend(autores_list)
    
    if todos_autores:
        conteo_autores = pd.Series(todos_autores).value_counts()
        
        print(f"📊 Top 10 congresistas más activos:")
        for i, (autor, count) in enumerate(conteo_autores.head(10).items(), 1):
            print(f"   {i:2d}. {autor}: {count} proyectos")
        
        # Análisis de colaboraciones
        print(f"\n🤝 ANÁLISIS DE COLABORACIONES")
        print("=" * 40)
        
        # Contar proyectos con múltiples autores
        proyectos_multiples = df_clean[df_clean['num_autores'] > 1]
        print(f"📈 Proyectos con múltiples autores: {len(proyectos_multiples)} ({len(proyectos_multiples)/len(df_clean)*100:.1f}%)")
        print(f"📊 Promedio de autores por proyecto: {df_clean['num_autores'].mean():.1f}")
        print(f"📊 Máximo de autores en un proyecto: {df_clean['num_autores'].max()}")
        
        # Análisis temporal
        if 'fecha_datetime' in df_clean.columns:
            print(f"\n📅 ANÁLISIS TEMPORAL")
            print("=" * 40)
            
            df_clean['dia_semana_num'] = df_clean['fecha_datetime'].dt.dayofweek
            proyectos_por_dia = df_clean.groupby('dia_semana').size()
            
            print(f"📊 Proyectos por día de la semana:")
            dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
            for dia, count in proyectos_por_dia.items():
                print(f"   {dia}: {count}")
            
            # Análisis por mes
            if df_clean['fecha_datetime'].dt.year.nunique() > 1:
                proyectos_por_mes = df_clean.groupby([df_clean['fecha_datetime'].dt.year, 
                                                    df_clean['fecha_datetime'].dt.month]).size()
                print(f"\n📊 Proyectos por mes:")
                for (año, mes), count in proyectos_por_mes.items():
                    print(f"   {año}-{mes:02d}: {count}")
    
    # Análisis de calidad de datos
    print(f"\n🔍 ANÁLISIS DE CALIDAD DE DATOS")
    print("=" * 40)
    
    print(f"📊 Completitud de datos:")
    for col in ['proyecto_limpio', 'fecha_limpia', 'titulo_limpio', 'partido_politico', 'tipo_proyecto']:
        if col in df_clean.columns:
            completitud = (1 - df_clean[col].isnull().sum() / len(df_clean)) * 100
            print(f"   {col}: {completitud:.1f}%")
    
    # Detectar posibles duplicados
    duplicados = df_clean.duplicated(subset=['proyecto_limpio'], keep=False)
    if duplicados.any():
        print(f"\n⚠️  Posibles duplicados detectados: {duplicados.sum()}")
        print("   Proyectos duplicados:")
        for proyecto in df_clean[duplicados]['proyecto_limpio'].unique():
            print(f"   - {proyecto}")
    else:
        print(f"\n✅ No se detectaron duplicados")
        
else:
    print("❌ No hay datos para análisis detallado")


In [ ]:
# Exportar datos limpios y resultados
if not df_clean.empty:
    
    # Guardar datos limpios
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    archivo_limpio = f'../data/proyectos_ley_limpios_{timestamp}.csv'
    df_clean.to_csv(archivo_limpio, index=False, encoding='utf-8-sig')
    print(f"💾 Datos limpios guardados en: {archivo_limpio}")
    
    # Guardar resumen en archivo de texto
    archivo_resumen = f'../analysis/resumen_analisis_{timestamp}.txt'
    with open(archivo_resumen, 'w', encoding='utf-8') as f:
        f.write("📊 RESUMEN DE ANÁLISIS - PROYECTOS DE LEY CONGRESO DEL PERÚ\n")
        f.write("=" * 70 + "\n\n")
        f.write(f"📅 Fecha de análisis: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
        f.write(f"📊 Total de proyectos: {resumen['total_proyectos']}\n")
        f.write(f"📅 Período: {resumen['fecha_inicio'].strftime('%d/%m/%Y')} - {resumen['fecha_fin'].strftime('%d/%m/%Y')}\n")
        f.write(f"👥 Promedio de autores por proyecto: {resumen['promedio_autores']:.1f}\n")
        f.write(f"🏆 Congresista más activo: {resumen['congresista_mas_activo']}\n\n")
        
        f.write("🏛️ PROYECTOS POR PARTIDO POLÍTICO:\n")
        for partido, count in resumen['proyectos_por_partido'].items():
            f.write(f"   {partido}: {count}\n")
        
        f.write(f"\n📋 PROYECTOS POR TIPO:\n")
        for tipo, count in resumen['proyectos_por_tipo'].items():
            f.write(f"   {tipo}: {count}\n")
        
        if resumen['proyectos_por_region']:
            f.write(f"\n🌍 PROYECTOS POR REGIÓN:\n")
            for region, count in resumen['proyectos_por_region'].items():
                f.write(f"   {region}: {count}\n")
    
    print(f"📄 Resumen guardado en: {archivo_resumen}")
    
    # Crear directorio de análisis si no existe
    os.makedirs('../analysis', exist_ok=True)
    os.makedirs('../visualizations', exist_ok=True)
    
    print(f"\n✅ Análisis completado exitosamente")
    print(f"📁 Archivos generados:")
    print(f"   - Datos limpios: {archivo_limpio}")
    print(f"   - Resumen: {archivo_resumen}")
    
else:
    print("❌ No hay datos para exportar")
